In [1]:

import multiprocessing
import os
import re
import signal
from math import ceil
from os.path import join

import numpy as np
import torch
from numpy.random import choice
from torchtext.data import Field, TabularDataset

import glob
from io import StringIO

%matplotlib inline

In [2]:
if not os.path.exists('model_data/data'):
    os.mkdir('model_data/data')

In [30]:
base_path = 'model_data/data/'

## sklearn

In [18]:
sk_glob = sorted(glob.glob('sklearn/*'))

In [19]:
samples = []
for file in sk_glob:
    with open(file) as f:
        samples.append(f.read())

In [35]:
len(samples)

243

In [31]:

with open(f'{base_path}sklearn.csv', 'w') as f:
    doc_text = "text\n" + "\n".join(['"' + sample.replace('\n', ' ') + '"' for sample in samples])
    f.write(doc_text)


## Caret

In [23]:
caret_glob = sorted(glob.glob('caret/*'))

In [25]:
caret_samples = []
for file in caret_glob:
    with open(file) as f:
        caret_samples.append(f.read())

In [26]:
len(caret_samples)

141

In [32]:

with open(f'{base_path}caret.csv', 'w') as f:
    doc_text = "text\n" + "\n".join(['"' + sample.replace('\n', ' ') + '"' for sample in samples])
    f.write(doc_text)


## Total

In [38]:
filenames = ['sklearn.csv', 'caret.csv']
with open(base_path + 'all_data.csv', 'w') as outfile:
    outfile.write("text\n")
    for fname in filenames:
        with open(base_path + fname) as infile:
            for line in infile:
                if line.strip() == 'text':
                    print("text")
                    continue
                outfile.write(line)

text
text
